# Simple CNN for Image Classification
-  dataset: MNIST
- TensorFlow: tf.layers, tf.Estimator
- Colab TensorBoard monitoring

In [0]:
import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
def make_model_fn(optimizer):
  
  def _conv_activate(input, filters, kernel_size, pool_size, strides, name):
    with tf.name_scope(name) as scope:
      conv = tf.layers.conv2d(inputs=input, 
                              filters=filters,
                              kernel_size=kernel_size,
                              padding="same",
                              activation=tf.nn.relu)
      
      pool = tf.layers.max_pooling2d(inputs=conv,
                                     pool_size=pool_size,
                                     strides=strides)
      
      return pool
  
  def _cnn(input_layer):
    pool1 = _conv_activate(input_layer, 32, [5, 5], [2, 2], 2, "conv1")
    pool2 = _conv_activate(pool1, 64, [5, 5], [2, 2], 2, "conv2")
    return pool2
    
  
  def _model_fn(features, labels, mode, params=None):
    """Model function for CNN."""
    
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layers
    conv_out = _cnn(input_layer)
    
    # Reshape
    embedding = tf.reshape(conv_out,
                            [-1, 7 * 7 * 64])
    
    # Dense Layer
    dense = tf.layers.dense(inputs=embedding,
                            units=1024,
                            activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(inputs=dense,
                                rate=0.4,
                                training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout,
                             units=10)

    # Compute predictions
    predicted_classes = tf.argmax(input=logits, axis=1)
    
    
    predictions = {
        'classes': predicted_classes[:, tf.newaxis],
        'probabilities': tf.nn.softmax(logits, name="softmax_tensor"),
        'logits': logits,
    }
    
    # PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
      return tf.estimator.EstimatorSpec(mode=mode,
                                        predictions=predictions)

    # Calculate Loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,
                                                  logits=logits)

    # Compute evaluation metrics
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])
    
    # EVAL
    if mode == tf.estimator.ModeKeys.EVAL:
      return tf.estimator.EstimatorSpec(mode,
                                        loss=loss, 
                                        eval_metric_ops=metrics)
    
    # TRAIN
    assert mode == tf.estimator.ModeKeys.TRAIN
    
    train_op = optimizer.minimize(loss, 
                                  global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
  
  return _model_fn

In [0]:
def make_train_input_fn(features,
                        labels,
                        batch_size=100,
                        num_epochs=None,
                        shuffle=True):

  train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": features},
                                                      y=labels,
                                                      batch_size=batch_size,
                                                      num_epochs=num_epochs,
                                                      shuffle=shuffle)
  return train_input_fn

def make_eval_input_fn(features,
                       labels):
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": features},
                                                     y=labels,
                                                     num_epochs=1,
                                                     shuffle=False)
  
  return eval_input_fn

In [0]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

In [0]:
MODEL_DIR="/tmp/mnist_convnet_model"
! rm -rf $MODEL_DIR

learning_rate=0.1
batch_size=100
max_steps=20000

In [7]:
# Create the Estimator
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_steps = 2000,
    #keep_checkpoint_max = 10,
    log_step_count_steps = 500,
    tf_random_seed=2020         # for reproducibility
)

train_spec = tf.estimator.TrainSpec(
    input_fn=make_train_input_fn(train_data,
                                 train_labels,
                                 batch_size=batch_size),
    max_steps=max_steps)

eval_spec = tf.estimator.EvalSpec(
    input_fn=make_eval_input_fn(eval_data,
                                eval_labels),
    steps=None,          # use complete eval set
    start_delay_secs=0,  # start immediately
    throttle_secs=10)    # minimum delay between evaluations


optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
classifier = tf.estimator.Estimator(
  model_fn=make_model_fn(optimizer),
  model_dir=MODEL_DIR,
  config=my_checkpointing_config
)

tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': 2020, '_save_summary_steps': 100, '_save_checkpoints_steps': 2000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 500, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd5af3d27b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop

({'accuracy': 0.9931, 'global_step': 20000, 'loss': 0.027608419}, [])

# Get ngrock and run it


In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-01 08:14:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.173.3.255, 34.206.9.96, 52.4.95.48, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.173.3.255|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.30M  15.5MB/s    in 0.9s    

2019-05-01 08:14:43 (15.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [9]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://c1994445.ngrok.io


# Start TensorBoard

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(MODEL_DIR)
)

# Stop Tensorboard

In [0]:
get_ipython().system_raw('pkill tensorboard')